In [1]:
import json
import replicate
import chromadb
from tqdm.auto import tqdm
import os

/home/aganap12/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from replicate.client import Client

replicate = Client(api_token="r8_IbZu0U3qUJ5ZC0TzzMm6xCJ6yU5UxVi16Ejo4")

In [3]:
client = chromadb.PersistentClient(path="./persistent-client")

In [4]:
def add_text_files_to_collection(collection, datasets_dir):
  """
  Adds text files from a directory to a collection with generated IDs.

  Args:
      collection: The collection object to add documents and IDs to.
      datasets_dir: The directory containing the text files.
  """
  text_files = [os.path.join(datasets_dir, filename) for filename in os.listdir(datasets_dir)
                if filename.endswith(".txt")]

  if not text_files:
    print(f"No text files found in directory: {datasets_dir}")
    return

  start_id = 1 
  for i, filename in enumerate(text_files):
    document_id = f"doc_{start_id + i}"

    with open(filename, 'r') as f:
      document_content = f.read()
        
    collection.add(documents=[document_content], ids=[document_id])

In [5]:
collection = client.get_or_create_collection(
        name="aws_blogs",
        metadata={"hnsw:space": "cosine"}
    )
datasets_dir = "../dataset/aws-case-studies-blogs-dataset" 
add_text_files_to_collection(collection, datasets_dir)

2024-04-18 08:23:55.694314647 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3782091, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-18 08:23:55.705141786 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3782092, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-18 08:23:55.736680102 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3782093, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-18 08:23:55.747219767 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3782094, index: 3, mask: {4, }, error code: 22 error msg: Invalid argument. Specify the n

In [6]:
import time 

query_string = "tell me about aws lambda"
start_time = time.time()
results = collection.query(
    query_texts=[query_string],
    n_results=2,
)
end_time = time.time()
retrieval_time = (end_time - start_time) * 1e6
print("Retrieval time:", retrieval_time, "microseconds")

Retrieval time: 1231719.2554473877 microseconds


In [7]:
successful_titles = ['\n'.join(results['documents'][0])]
successful_titles[:10]

['In addition to better compliance with regulations through improved transparency, the Purple IT team has improved and accelerated software development processes using AWS.\nAWS Lambda is a serverless, event-driven compute service that lets you run code for virtually any type of application or backend service without provisioning or managing servers. You can trigger Lambda from over 200 AWS services and software as a service (SaaS) applications, and only pay for what you use.File processing\xa0Stream processing\xa0Web applications\xa0IoT backends\xa0Mobile backends.  \nFrançais\n Benefits of AWS\nJan Červinka Director of Engineering, Purple Technology \nEspañol\n Amazon EC2\nThe Czech-based company builds apps that complement online trading platforms and support the changing and demanding needs of brokers. Purple’s solution enables tens of thousands of clients to trade many billions of dollars of assets each month.\n日本語\nIn addition, brokers and traders must comply with rules that chan

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = f"{query_string}\n\n{top_k_documents[0]}"
input_ids = tokenizer.encode(prompt, return_tensors="pt")